In [1]:
# test 
# basic model
# date : 230908

In [2]:
import argparse
import os
import torch

In [3]:
# from attrdict import AttrDict

In [4]:
import sys
sys.path.append('/home/gpuadmin/dev/Trajectory_Prediction/traffino')

In [5]:
from data.loader_basic import data_loader # change
from model_basic import TrajectoryGenerator # change
from losses import displacement_error, final_displacement_error
from traffino.utils import relative_to_abs, get_dset_path

In [6]:
parser = argparse.ArgumentParser()

In [7]:
from typing import Tuple
class CreateArg():

    def __init__(self):

        # test
        self.model_path = '/home/gpuadmin/dev/Trajectory_Prediction/traffino/output_4/basic_output' # .pt 파일 불러올 경로
        self.num_samples = 20 # type=int
        self.dset_type = "test" # type=str   

        # Dataset options
        self.dataset_name = 'waterloo'
        self.delim = '\t'
        self.loader_num_workers = 4
        self.obs_len = 8
        self.pred_len = 4     ################# check
        self.skip = 1
        # Optimization
        self.batch_size = 8     ################# check
        self.num_iterations = 1000 # 10000
        self.num_epochs = 50 # 200                     
        # Model Options
        self.embedding_dim = 64
        self.num_layers = 1
        self.dropout = 0.0
        self.batch_norm = 0 
        self.mlp_dim = 1024
        # self.default_backbone= 'resnet18'
                                    
        # Generator Options
        self.encoder_h_dim_g = 64
        self.decoder_h_dim_g = 128
        self.noise_dim : Tuple[int] = (0, 0) # default=None # type=int_tuple
        self.noise_type = 'gaussian'
        self.noise_mix_type = 'ped'
        self.clipping_threshold_g = 0 # type=float
        self.g_learning_rate = 5e-4 # type=float 
        self.g_steps = 1

        # Pooling Options
        self.pooling_type = 'pool_net' 
        self.pool_every_timestep = 1 # type=bool_flag

        # Pool Net Option
        self.bottleneck_dim = 1024 # type=int

        # Social Pooling Options
        self.neighborhood_size = 1024 # type=float
        self.grid_size = 8 # type=int

        # Discriminator Options
        self.d_type = 'local' # type=str
        self.encoder_h_dim_d = 64 # type=int
        self.d_learning_rate = 5e-4 # type=float
        self.d_steps = 2 # type=int        
        self.clipping_threshold_d = 0 # type=float  

        # Loss Options
        self.l2_loss_weight = 0 # type=float 
        self.best_k = 1 # type=int 

        # Output
        self.output_dir = os.getcwd()+'/basic_output'   # 평가 단계에선 안쓰임
        self.print_every = 5 # type=int
        self.checkpoint_every = 100 # type=int
        self.checkpoint_name = 'checkpoint_basic' 
        self.checkpoint_start_from = None
        self.restore_from_checkpoint = 1 # type=int
        self.num_samples_check = 5000 # type=int        

        # Misc
        self.use_gpu = 1 # type=int
        self.timing = 0 # type=int
        self.gpu_num = "1" # type=str   


args = CreateArg() 

In [8]:
def get_generator(checkpoint):
    args_ = args
    generator = TrajectoryGenerator(
        obs_len=args_.obs_len,
        pred_len=args_.pred_len,
        embedding_dim=args_.embedding_dim,
        encoder_h_dim=args_.encoder_h_dim_g,
        decoder_h_dim=args_.decoder_h_dim_g,
        mlp_dim=args_.mlp_dim,
        num_layers=args_.num_layers,
        noise_dim=args_.noise_dim,
        noise_type=args_.noise_type,
        noise_mix_type=args_.noise_mix_type,
        pooling_type=args_.pooling_type,
        pool_every_timestep=args_.pool_every_timestep,
        dropout=args_.dropout,
        bottleneck_dim=args_.bottleneck_dim,
        neighborhood_size=args_.neighborhood_size,
        # grid_size=args_.grid_size,
        batch_norm=args_.batch_norm)
    generator.load_state_dict(checkpoint['g_state'])
    generator.cuda()
    generator.train()
    return generator

In [9]:
def evaluate_helper(error, seq_start_end):
    sum_ = 0
    error = torch.stack(error, dim=1)

    for (start, end) in seq_start_end:
        start = start.item()
        end = end.item()
        _error = error[start:end]
        _error = torch.sum(_error, dim=0)
        _error = torch.min(_error)
        sum_ += _error
    return sum_

In [10]:
def evaluate(args, loader, generator, num_samples):
    ade_outer, fde_outer = [], []
    total_traj = 0
    with torch.no_grad():
        for batch in loader:
            batch = [tensor.cuda() for tensor in batch]
            (obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel,
             non_linear_ped, loss_mask, seq_start_end) = batch
            



            ade, fde = [], []
            total_traj += pred_traj_gt.size(1)

            for _ in range(num_samples):
                pred_traj_fake_rel = generator(
                    obs_traj, obs_traj_rel, seq_start_end,
                )
                pred_traj_fake = relative_to_abs(
                    pred_traj_fake_rel, obs_traj[-1]
                )
                ade.append(displacement_error(
                    pred_traj_fake, pred_traj_gt, mode='raw'
                ))
                fde.append(final_displacement_error(
                    pred_traj_fake[-1], pred_traj_gt[-1], mode='raw'
                ))

            ade_sum = evaluate_helper(ade, seq_start_end)
            fde_sum = evaluate_helper(fde, seq_start_end)

            ade_outer.append(ade_sum)
            fde_outer.append(fde_sum)
        ade = sum(ade_outer) / (total_traj * args.pred_len)
        fde = sum(fde_outer) / (total_traj)
        return ade, fde


In [11]:
if os.path.isdir(args.model_path):
    filenames = os.listdir(args.model_path)
    print(filenames)
    filenames.sort()
    paths = [
        os.path.join(args.model_path, file_) for file_ in filenames
    ]
else:
    paths = [args.model_path]

['checkpoint_basic_with_model.pt']


In [12]:
print(args.model_path)
print(filenames)
print(paths)
print(f"args.pred_len:{args.pred_len}")
print(f"args.batch_size:{args.batch_size}")
print(f"batch_size: {args.batch_size}")

/home/gpuadmin/dev/Trajectory_Prediction/traffino/output_4/basic_output
['checkpoint_basic_with_model.pt']
['/home/gpuadmin/dev/Trajectory_Prediction/traffino/output_4/basic_output/checkpoint_basic_with_model.pt']
args.pred_len:4
args.batch_size:8
batch_size: 8


In [13]:
for path in paths:
    print(path)
    checkpoint = torch.load(path)
    # print(f"checkpoint: {checkpoint}\n")
    generator = get_generator(checkpoint)
    #_args = AttrDict(checkpoint['args'])
    path = '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/test'
    _, loader = data_loader(args, path)
    ade, fde = evaluate(args, loader, generator, args.num_samples)
    print('Dataset: {}, Pred Len: {}, ADE: {:.2f}, FDE: {:.2f}'.format(
        args.dataset_name, args.pred_len, ade, fde))


/home/gpuadmin/dev/Trajectory_Prediction/traffino/output_4/basic_output/checkpoint_basic_with_model.pt
input_dim:1088
['0785_prep.txt', '0784_prep.txt']
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/test/0784_prep.txt
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/test/0785_prep.txt
Dataset: waterloo, Pred Len: 4, ADE: 5.36, FDE: 8.94
